In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from pylab import rcParams
from scipy.ndimage.interpolation import rotate, zoom
import scipy as sp
import cv2
%matplotlib inline
import sys
from tqdm import tqdm
import argparse
from IPython.display import clear_output

sys.path.append("/home/msmith/misc/pcaRotate/") # Helper function
from pcaRotate import main as rotate
from pcaRotate import getRedHSV
from pylab import rcParams
rcParams["figure.figsize"] = 30,25
#np.random.seed(1006)
import pickle

def saveList(wp,lst):
    with open(wp, 'wb') as fp:
        pickle.dump(lst, fp)

def show(wp,orig,mask,maskRot,red,thresh,croppedHead):
    
    plt.suptitle(wp)
    
    plt.subplot(161)
    plt.imshow(orig)
    plt.subplot(162)
    plt.imshow(mask)
    plt.subplot(163)
    plt.imshow(maskRot,cmap=cm.gray)
    plt.subplot(164)
    plt.imshow(red,cmap=cm.gray)
    plt.subplot(165)
    plt.imshow(thresh,cmap=cm.gray)
    plt.subplot(166)
    plt.imshow(croppedHead)
    plt.show()

In [ ]:
if __name__ == "__main__": 
    
    ss = 0
    save = 1
    if ss == 1:
        maskPath = "m1_ss_"
        dstPath = "head_ss_"
        ext = "ss_"
    else:
        maskPath = "m1_"
        dstPath = "head_"
        ext = "[0-9]"
    
    imgPaths = glob.glob("/home/msmith/kaggle/whale/imgs/whale*/m1_{0}*".format(ext))
    testPaths = glob.glob("/home/msmith/kaggle/whale/imgs/test*/m1_{0}*".format(ext))
    print("Make sure image is in RGB order not BGR")
    imgPaths.sort()
    imgPaths += testPaths

    cantSave = []
    linAlgError = []
    outSize = 500
    w = h = outSize
    count = 0
    for path in tqdm(imgPaths):
        
        try:
            orig, mask = [cv2.imread(x)[:,:,::-1] for x in [path.replace(maskPath,"w1_"),path]]
            croppedHead, maskRot, red, thresh, fail = rotate(orig=orig,mask=mask,ellipseThresh=8,hThr=[0.06,0.95],vThr=0.1,cntThresh=0.000001,pad=50,outSize=outSize)
           
            wp = path.replace(maskPath,dstPath)

            if fail != 0:
                print("couldn't save {0}. Fail type = {1}".format(wp,fail))
                cantSave.append([wp,fail])
                
            else:
                croppedHead = cv2.resize(croppedHead,(w,h),interpolation=cv2.INTER_LINEAR)
                if save == 1:
                    cv2.imwrite(wp,croppedHead)
                    #print("Saved in {0}.".format(wp))

                if np.random.uniform() < 0.01:
                    show(wp,orig,mask,maskRot,red,thresh,croppedHead)
                    count += 1

                if count == 10:
                    clear_output()
                    count = 0
        except np.linalg.linalg.LinAlgError as err:
            linAlgError.append(path)
    saveList("imgCropFails.txt",cantSave)

  0%|          | 0/11468 [00:00<?, ?it/s]

Make sure image is in RGB order not BGR


  0%|          | 14/11468 [00:15<2:23:25,  1.33it/s]